In [ ]:
#%%
import geopandas as gpd
from geopandas import GeoDataFrame, GeoSeries
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
# matplotlib inline
#%pip install seaborn
# import seaborn as sns
from shapely.geometry import Point, Polygon
import numpy as np
%pip install googlemaps
import googlemaps
from datetime import datetime, timedelta
plt.rcParams["figure.figsize"] = [8,6]
import pandas as pd
#%pip install  simplekml
import simplekml
import os
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from rasterio.crs import CRS
# %pip install rioxarray
# %pip install earthpy
import rioxarray as rxr
import earthpy as et
import rasterio
from rasterio.plot import show
import fiona
import glob
from shapely.geometry import box
# import seaborn as sns
# import seaborn as sns
import os
#from whitebox.whitebox_tools import WhiteboxTools
import rasterio
from rasterio.plot import show
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import folium
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc


## **Problem 2:**  
Consider the ANOVA model given on page 64 of Montgomery's textbook (Equations (3.1) and (3.2))  
1. State the statistical assumptions which are usually made when working with this model.  
2. Create a Python or Matlab simulation where you specify treatment levels, then draw random numbers appropriately and run a Monte Carlo simulation of such an experiment.  
3. Depending on the assumptions you make you should be able to solve the ANOVA problem analytically and then compare the solution and performance with the Monte Carlo result.Explain what you see.  
   

<span style="color:#fff; font-family: 'Babas neue'; font-size: 1.5em;">Solution part a</span>  
The assumptions for analyzing variance of random variables:  
- Error is normally distributed  
- Error is independently distributed  
- Mean error is zero and variance is $\sigma^{2}$
- Random variable is normally and independently distributed with mean $\mu + \tau_{i}$ (difference in treatment mean) and variance $\sigma^{2}$

<span style="color:#fff; font-family: 'Babas neue'; font-size: 1.5em;">Solution part b and c</span>  
I have considered the problem of comparing the effect of varying planting days on sseason length of corn (GDD to maturity 2600). Planting date is randomly sselected  from 4 15 day periods from Apr 15th to june 15th for 1000 simulation. These simulations calculate the days for corn to collect 2600 GDD from the historical average temperature. The temperature is the mean daily from 1981 to 2023 scraped from ACIS-RCC for Purdue.


In [ ]:
import requests
# GPS coordinates of Purdue ACRE farm
lat = 40.4742259
lon = -86.9975974
# PRISM goes back to 1981
sdate = "1981-01-01" #start date for historical data
edate = "2023-12-31" #end date for historical data
# Make ACIS API request
w = requests.post('http://data.rcc-acis.org/GridData', json={
    "loc": f"{lon}, {lat}",
    "sdate": sdate,
    "edate": edate,
    "grid": "21",   # "21" is PRISM
    "elems": [
    # "maxt" is maximum temperature at interval daily 
    #in degree Faherenheit and precipitation in inch
        { "name":"maxt", "interval":"dly", "units":"degreeF" }, 
        { "name":"mint", "interval":"dly", "units":"degreeF" },
        { "name":"pcpn", "interval":"dly", "units":"inch" }
    ]  
})
# Parse the JSON response to a Python datatypes
w = w.json()
#use for loop to print the first 5 items in the retrieved weather data
for item in w['data'][0:5]: 
    print("the retrived weather data: ",item)  

In [ ]:
# Convert the raw Python map to a Pandas DataFrame
w = pd.DataFrame(w['data'], columns=['date', 'maxt', 'mint', 'pcpn'])
w['date'] = pd.to_datetime(w['date'])
w.set_index('date', inplace=True)


In [ ]:
t_max=84 #maximum temperature for corn growth
t_base=50 #base temperature for corn growth
w["maxt_gd"]=w["maxt"] #create a new column for max temperature for column operations
w["mint_gd"]=w["mint"]#create a new column for min temperature for column operations
w.loc[w.maxt_gd > t_max, 'maxt_gd'] = t_max #set the max temperature to 84 if it is greater than 84

# Save the daily GDU totals back to the DataFrame as a new column
w['gdu'] = (w.maxt_gd + w.mint_gd) / 2 - t_base
# Negative GDUs are assumed to be zero ... that is a colder day doesn't reduce the growth
w.loc[w.gdu < 0, 'gdu'] = 0 
w.describe()

In [ ]:
w_copy = w.copy()
w_copy['avgt'] = (w_copy.mint + w_copy.maxt) / 2

In [ ]:
# Select the rows to modify and then modify them.
w_copy.loc[w_copy.mint < 0, 'mint'] = 0


In [ ]:
# Day of year: Jan 1 -> 0, Jan 2 -> 1, Jan 3 -> 2, ... without regard the actual year
byDay = w.groupby(w.index.dayofyear)
byDay.head(20)


In [ ]:
#Concatenate the mean of max temperature, min temperature, GDU and precipitation by day of year
byDate_df_mean=pd.concat([byDay.maxt.mean(), byDay.mint.mean(), byDay.gdu.mean(),byDay.pcpn.mean()], axis='columns')
byDate_df_mean["gdu"]= byDate_df_mean["gdu"].replace({'0':np.nan, 0:np.nan})#replace 0 with NaN
byDate_df_mean["gdu_cumsum"]=byDate_df_mean["gdu"].cumsum()


In [ ]:
monthday=[]
for day in range(1,367):
    monthday_dt=datetime(2020, 1, 1) + timedelta(day- 1)
    something="2020-"+str(monthday_dt.month)+"-"+str(monthday_dt.day)
    monthday.append(something)

monthday


# month=[]
# for day in range(1,367):
#     monthday_dt=datetime(2020, 1, 1) + timedelta(day- 1)
#     something=str(monthday_dt.month)
#     month.append(something)

# print(len(monthday))
# print(month)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] =600
import matplotlib.dates as mdates
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.ticker import NullFormatter



# You can select a column out of the "GroupBy" before processing them
# You can use the resulting DataFrame just like before, e.g., plotting.
ax = plt.subplot(3, 1, 1)

plt.errorbar(monthday, byDay.maxt.mean(), byDay.maxt.std(), linewidth =0.5)
plt.legend(['Max Temp'])
plt.ylabel('Temperature (F)')
plt.title('Error bar plot for Average Max daily temperature since 1981')
plt.xticks(rotation=25)
# ax.set_xlim([monthday[0], monthday[-1]])

xfmt = mdates.DateFormatter('%b')
months = mdates.MonthLocator()
ax.xaxis.set_major_locator(months)
ax.xaxis.set_minor_locator(MonthLocator(bymonthday=15))
#ax.xaxis.set_major_formatter(xfmt)
ax.xaxis.set_major_formatter(DateFormatter('%b'))
ax.xaxis.set_minor_formatter(NullFormatter())
# Hide the last tick label if it is January
ticks = ax.get_xticks()
ax.set_xticks(ticks[:-1])  # Remove the last tick
# Now we select the second slice. Notice we share the x-axis. This is so when you interact with the either plot
# the other plot is automatically adjusted to the same scale.
plt.subplot(3, 1, 2, sharex=ax)
plt.plot(byDay.groups.keys(), byDay.gdu.sum(), linewidth=0.5)
plt.ylabel('GDD (Tbase=50F)')
plt.xticks(rotation=25)

plt.subplot(3, 1, 3, sharex=ax)
plt.plot( byDay.groups.keys(),byDay.pcpn.sum(), linewidth=0.5)
plt.ylabel('Rain (in)')
plt.xticks(rotation=25)

plt.xlabel('Day of year')
plt.title('Mean daily GDU since 1981 ')

plt.tight_layout()
plt.show()

In [ ]:
hist_weather=w_copy.copy()
byDate_df_mean["date"]=monthday
byDate_df_mean['date'] = pd.to_datetime(byDate_df_mean['date'])
byDate_df_mean.set_index('date', inplace=True)
byDate_df_mean[byDate_df_mean.index > "2020-04-15"]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

list_ExperimentDfs=[]

# Define a function to calculate GDD accumulation from a given start date
def calculate_gdd_accumulation(start_date, gdd_data, target_gdd):
    accumulated_gdd = 0
    days_count = 0
    for index, row in gdd_data.iterrows():
        if index >= start_date:
            accumulated_gdd += row['gdu']
            days_count += 1
            if accumulated_gdd >= target_gdd:
                break
    return days_count


def run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd):
    simulation_results = []

    for _ in range(num_simulations):
        # Calculate the number of days between the start and end date
        num_days = (end_date - start_date).days

        # Randomly select a day within this range
        random_day_within_range = np.random.randint(0, num_days + 1)

        # Generate the random planting date within the specified range
        random_planting_date = start_date + pd.Timedelta(days=random_day_within_range)

        # Calculate the number of days to reach the target GDD
        days_to_target_gdd = calculate_gdd_accumulation(random_planting_date, byDate_df_mean, target_gdd)

        # Store the result
        simulation_results.append((random_planting_date, days_to_target_gdd))

    # Convert the simulation results to a DataFrame
    simulation_results_df = pd.DataFrame(simulation_results, columns=['Planting Date', 'Days to Target GDD'])
    return simulation_results_df

# Example usage
num_simulations = 1000
start_date = pd.Timestamp('2020-04-15')
end_date = pd.Timestamp('2020-04-30')

target_gdd = 2600

# Assuming 'byDate_df_mean' is your DataFrame containing the GDD data
# Replace 'byDate_df_mean' with your actual DataFrame variable

# Run the simulation
simulation_results_df_4 = run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd)

# Create a list of dates corresponding to the number of days since planting
simulation_results_df_4['Date to Target GDD'] = simulation_results_df_4['Planting Date'] + pd.to_timedelta(simulation_results_df_4['Days to Target GDD'], unit='D')

#append the resulting dfs to a list for more analysis
list_ExperimentDfs.append(simulation_results_df_4)


# Convert dates to strings for categorical plotting
simulation_results_df_4['Date to Target GDD'] = simulation_results_df_4['Date to Target GDD'].dt.strftime('%m/%d')

# Plot the histogram with categorical x-axis
plt.figure(figsize=(10, 6))
simulation_results_df_4['Date to Target GDD'].value_counts().sort_index().plot(kind='bar', edgecolor='k', alpha=0.7)
plt.xlabel('Date to Reach 2600 GDD')
plt.ylabel('Frequency')
plt.title('Monte Carlo Simulation of Dates to Reach 2600 GDD')

# Rotate the date labels for better visibility
plt.gcf().autofmt_xdate()

plt.savefig("GDD_MC_0415-0430.png", dpi=600)
plt.show()

# Calculate the average number of days to reach the target GDD
average_days_to_target_gdd = simulation_results_df_4['Days to Target GDD'].mean()
print(f'Average number of days to reach 2600 GDD: {average_days_to_target_gdd:.2f} days')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

list_ExperimentDfs=[]

# Define a function to calculate GDD accumulation from a given start date
def calculate_gdd_accumulation(start_date, gdd_data, target_gdd):
    accumulated_gdd = 0
    days_count = 0
    for index, row in gdd_data.iterrows():
        if index >= start_date:
            accumulated_gdd += row['gdu']
            days_count += 1
            if accumulated_gdd >= target_gdd:
                break
    return days_count


def run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd):
    simulation_results = []

    for _ in range(num_simulations):
        # Calculate the number of days between the start and end date
        num_days = (end_date - start_date).days

        # Randomly select a day within this range
        random_day_within_range = np.random.randint(0, num_days + 1)

        # Generate the random planting date within the specified range
        random_planting_date = start_date + pd.Timedelta(days=random_day_within_range)

        # Calculate the number of days to reach the target GDD
        days_to_target_gdd = calculate_gdd_accumulation(random_planting_date, byDate_df_mean, target_gdd)

        # Store the result
        simulation_results.append((random_planting_date, days_to_target_gdd))

    # Convert the simulation results to a DataFrame
    simulation_results_df = pd.DataFrame(simulation_results, columns=['Planting Date', 'Days to Target GDD'])
    return simulation_results_df

# Example usage
num_simulations = 1000
start_date = pd.Timestamp('2020-05-01')
end_date = pd.Timestamp('2020-05-15')

target_gdd = 2600

# Assuming 'byDate_df_mean' is your DataFrame containing the GDD data
# Replace 'byDate_df_mean' with your actual DataFrame variable

# Run the simulation
simulation_results_df_3 = run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd)

# Create a list of dates corresponding to the number of days since planting
simulation_results_df_3['Date to Target GDD'] = simulation_results_df_3['Planting Date'] + pd.to_timedelta(simulation_results_df_3['Days to Target GDD'], unit='D')

#append the resulting dfs to a list for more analysis
list_ExperimentDfs.append(simulation_results_df_3)


# Convert dates to strings for categorical plotting
simulation_results_df_3['Date to Target GDD'] = simulation_results_df_3['Date to Target GDD'].dt.strftime('%m/%d')

# Plot the histogram with categorical x-axis
plt.figure(figsize=(10, 6))
simulation_results_df_3['Date to Target GDD'].value_counts().sort_index().plot(kind='bar', edgecolor='k', alpha=0.7)
plt.xlabel('Date to Reach 2600 GDD')
plt.ylabel('Frequency')
plt.title('Monte Carlo Simulation of Dates to Reach 2600 GDD')

# Rotate the date labels for better visibility
plt.gcf().autofmt_xdate()

plt.savefig("GDD_MC_0501-0515.png", dpi=600)
plt.show()

# Calculate the average number of days to reach the target GDD
average_days_to_target_gdd = simulation_results_df_3['Days to Target GDD'].mean()
print(f'Average number of days to reach 2600 GDD: {average_days_to_target_gdd:.2f} days')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

list_ExperimentDfs=[]

# Define a function to calculate GDD accumulation from a given start date
def calculate_gdd_accumulation(start_date, gdd_data, target_gdd):
    accumulated_gdd = 0
    days_count = 0
    for index, row in gdd_data.iterrows():
        if index >= start_date:
            accumulated_gdd += row['gdu']
            days_count += 1
            if accumulated_gdd >= target_gdd:
                break
    return days_count


def run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd):
    simulation_results = []

    for _ in range(num_simulations):
        # Calculate the number of days between the start and end date
        num_days = (end_date - start_date).days

        # Randomly select a day within this range
        random_day_within_range = np.random.randint(0, num_days + 1)

        # Generate the random planting date within the specified range
        random_planting_date = start_date + pd.Timedelta(days=random_day_within_range)

        # Calculate the number of days to reach the target GDD
        days_to_target_gdd = calculate_gdd_accumulation(random_planting_date, byDate_df_mean, target_gdd)

        # Store the result
        simulation_results.append((random_planting_date, days_to_target_gdd))

    # Convert the simulation results to a DataFrame
    simulation_results_df = pd.DataFrame(simulation_results, columns=['Planting Date', 'Days to Target GDD'])
    return simulation_results_df

# Example usage
num_simulations = 1000
start_date = pd.Timestamp('2020-05-15')
end_date = pd.Timestamp('2020-05-31')

target_gdd = 2600

# Assuming 'byDate_df_mean' is your DataFrame containing the GDD data
# Replace 'byDate_df_mean' with your actual DataFrame variable

# Run the simulation
simulation_results_df_2 = run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd)

# Create a list of dates corresponding to the number of days since planting
simulation_results_df_2['Date to Target GDD'] = simulation_results_df_2['Planting Date'] + pd.to_timedelta(simulation_results_df_2['Days to Target GDD'], unit='D')

#append the resulting dfs to a list for more analysis
list_ExperimentDfs.append(simulation_results_df_2)


# Convert dates to strings for categorical plotting
simulation_results_df_2['Date to Target GDD'] = simulation_results_df_2['Date to Target GDD'].dt.strftime('%m/%d')

# Plot the histogram with categorical x-axis
plt.figure(figsize=(10, 6))
simulation_results_df_2['Date to Target GDD'].value_counts().sort_index().plot(kind='bar', edgecolor='k', alpha=0.7)
plt.xlabel('Date to Reach 2600 GDD')
plt.ylabel('Frequency')
plt.title('Monte Carlo Simulation of Dates to Reach 2600 GDD')

# Rotate the date labels for better visibility
plt.gcf().autofmt_xdate()

plt.savefig("GDD_MC_0515-0531.png", dpi=600)
plt.show()

# Calculate the average number of days to reach the target GDD
average_days_to_target_gdd = simulation_results_df_2['Days to Target GDD'].mean()
print(f'Average number of days to reach 2600 GDD: {average_days_to_target_gdd:.2f} days')

In [ ]:
simulation_results_df_2["Date to Target GDD"].value_counts().sort_index()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

list_ExperimentDfs=[]

# Define a function to calculate GDD accumulation from a given start date
def calculate_gdd_accumulation(start_date, gdd_data, target_gdd):
    accumulated_gdd = 0
    days_count = 0
    for index, row in gdd_data.iterrows():
        if index >= start_date:
            accumulated_gdd += row['gdu']
            days_count += 1
            if accumulated_gdd >= target_gdd:
                break
    return days_count
# # Define the target GDD for corn
# target_gdd = 2600

# # Perform Monte Carlo simulation
# simulation_results = []
# num_simulations = 10

# for _ in range(num_simulations):
#     # Define the start and end date for the range
#     start_date = pd.Timestamp(year=byDate_df_mean.index.year.min(), month=4, day=15)
#     end_date = pd.Timestamp(year=byDate_df_mean.index.year.min(), month=5, day=1)

#     # Calculate the number of days between the start and end date
#     num_days = (end_date - start_date).days

#     # Randomly select a day within this range
#     random_day_within_range = np.random.randint(0, num_days + 1)

#     # Generate the random planting date within the specified range
#     random_planting_date = start_date + pd.Timedelta(days=random_day_within_range)
#     print("Selected planting date: ",random_planting_date)


#     # Calculate the number of days to reach the target GDD
#     days_to_target_gdd = calculate_gdd_accumulation(random_planting_date, byDate_df_mean, target_gdd)
#     print("Calculated maturity: ",days_to_target_gdd)

#     # Store the result
#     simulation_results.append((random_planting_date, days_to_target_gdd))

# # Convert the simulation results to a DataFrame
# simulation_results_df = pd.DataFrame(simulation_results, columns=['Planting Date', 'Days to Target GDD'])

def run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd):
    simulation_results = []

    for _ in range(num_simulations):
        # Calculate the number of days between the start and end date
        num_days = (end_date - start_date).days

        # Randomly select a day within this range
        random_day_within_range = np.random.randint(0, num_days + 1)

        # Generate the random planting date within the specified range
        random_planting_date = start_date + pd.Timedelta(days=random_day_within_range)

        # Calculate the number of days to reach the target GDD
        days_to_target_gdd = calculate_gdd_accumulation(random_planting_date, byDate_df_mean, target_gdd)

        # Store the result
        simulation_results.append((random_planting_date, days_to_target_gdd))

    # Convert the simulation results to a DataFrame
    simulation_results_df = pd.DataFrame(simulation_results, columns=['Planting Date', 'Days to Target GDD'])
    return simulation_results_df

# Example usage
num_simulations = 1000
start_date = pd.Timestamp('2020-06-01')
end_date = pd.Timestamp('2020-06-15')

target_gdd = 2600

# Assuming 'byDate_df_mean' is your DataFrame containing the GDD data
# Replace 'byDate_df_mean' with your actual DataFrame variable

# Run the simulation
simulation_results_df = run_gdd_simulation(num_simulations, start_date, end_date, byDate_df_mean, target_gdd)

# Create a list of dates corresponding to the number of days since planting
simulation_results_df['Date to Target GDD'] = simulation_results_df['Planting Date'] + pd.to_timedelta(simulation_results_df['Days to Target GDD'], unit='D')

#append the resulting dfs to a list for more analysis
list_ExperimentDfs.append(simulation_results_df)


# Convert dates to strings for categorical plotting
simulation_results_df['Date to Target GDD'] = simulation_results_df['Date to Target GDD'].dt.strftime('%m/%d')

# Plot the histogram with categorical x-axis
plt.figure(figsize=(10, 6))
simulation_results_df['Date to Target GDD'].value_counts().sort_index().plot(kind='bar', edgecolor='k', alpha=0.7)
plt.xlabel('Date to Reach 2600 GDD')
plt.ylabel('Frequency')
plt.title('Monte Carlo Simulation of Dates to Reach 2600 GDD')

# Rotate the date labels for better visibility
plt.gcf().autofmt_xdate()

plt.savefig("GDD_MC_0601-0615.png", dpi=600)
plt.show()

# Calculate the average number of days to reach the target GDD
average_days_to_target_gdd = simulation_results_df['Days to Target GDD'].mean()
print(f'Average number of days to reach 2600 GDD: {average_days_to_target_gdd:.2f} days')


### the boxplot illustrates the difference in the average number of days taken by corn to store 2600 GDD when planted in 4 different date ranges. The Corn planted in May 1-15 seems to take the least time.The corn planted in lat may uses the second least number of days but is prone to variability due to weather. Early july crops take much longer.

In [ ]:
import datetime

# Define the starting date
start_date = datetime.datetime(2020, 6, 13)

# Calculate the date 200 days after the starting date
end_date = start_date + datetime.timedelta(days=200)

print(end_date.strftime('%Y-%m-%d'))
import pandas as pd
import matplotlib.pyplot as plt


# Combine the dataframes into a single dataframe with a column indicating the source dataframe
simulation_results_df['Source'] = '0601-0615'
simulation_results_df_2['Source'] = '0515-0531'
simulation_results_df_3['Source'] = '0501-0515'
simulation_results_df_4['Source'] = '0415-0430'

combined_df = pd.concat([simulation_results_df_4, simulation_results_df_3, simulation_results_df_2, simulation_results_df])

# Plot the boxplot
plt.figure(figsize=(4, 3))
combined_df.boxplot(column='Days to Target GDD', by='Source', grid=False)

# Calculate and plot the means
means = combined_df.groupby('Source')['Days to Target GDD'].mean()
plt.plot(range(1, len(means) + 1), means, color='deeppink', marker='^', linestyle='-', linewidth=1, markersize=5, label='Mean')


plt.title('Boxplot of Days to Target GDD ')
plt.suptitle('')  # Suppress the default title
plt.xlabel('planting date range')
plt.ylabel('Days to Target GDD')
plt.show()


In [ ]:

combined_df_anova=combined_df.copy()
combined_df_anova.columns=combined_df_anova.columns.str.replace(" ","")
combined_df_anova.head()
#yield_gott93.columns = yield_gott93.columns.str.replace(' ', '') 


In [ ]:

#the anova analysis of the effect of planting date range on the number of days required for corn to accumulate 2600 gdd controlled by historical weather.
model = ols('DaystoTargetGDD ~ C(Source)', data=combined_df_anova).fit()
print("model parameters",model.summary())
# Perform one-way ANOVA, typ=2 is for both direction testin gof hypothesis.
anova_results = sm.stats.anova_lm(model, typ=2)
print(anova_results)


### The very high F stat value and its low probability (>0.05) shows the significant difference in the number of days taken by corn to gather 2600 GDD when planted in late april, vs June and early july.

The effect size of the varibles is used to calculate how much variation in the samples is caused due to the treatments. According to Cohen  
- effect size <0.2 small effect size (not much significance)
-  0.2< effect size <0.8 medium effect size
-  effect size > 0.8 large effect (significant)

In [ ]:

def cohend(data1, data2):
    from numpy import mean
    from numpy import var
    from math import sqrt
	# calculate the size of samples
    n1, n2 = len(data1), len(data2)
	# calculate the variance of the samples
    s1, s2 = var(data1, ddof=1), var(data2, ddof=1)
	# calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
	# calculate the means of the samples
    u1, u2 = mean(data1), mean(data2)
	# calculate the effect size
    return (u1 - u2) / s
for row in range(len(list(combined_df_anova["Source"].unique()))):
    df=pd.DataFrame()
    df1=pd.DataFrame()
    groupeddf=combined_df_anova.groupby("Source")
    df=groupeddf.get_group(list(combined_df_anova["Source"].unique())[row])
    #print(df.head())
    d1=df["DaystoTargetGDD"]
    df1=groupeddf.get_group(list(combined_df_anova["Source"].unique())[row+1])
    d2=df1["DaystoTargetGDD"]
    effectsize=cohend(d1,d2)
    print("effect size:  ", effectsize)
import statsmodels.stats.power as smp

# Set parameters
effect_size = 3.38  # Desired effect size (e.g., Cohen's d). 
alpha = 0.05       # Significance level
k_groups = 4       # Number of groups in the ANOVA
nobs = 3997        # Total sample size (across all groups)

# Create an FTestAnovaPower object
power_analysis = smp.FTestAnovaPower()

# Calculate power
power = power_analysis.power(effect_size, nobs, alpha, k_groups)

print(f"Power: {power}")




**Dont worry about the error here; its by design.**  
### The Effect size of all three pairs are high according to Cohen's range of effect sizes. This suggests the planting dates are significant in deciding the number of days for corn to collect 2600 GDD. The effect size supports the ANOVA analysis.

### `The graph below shows the GDD though out the years from 1981 to 2023.The avrage GDD is in green. The GDD is calculated from May1st corn (2600 GDD).`

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates



# Planting date
planting_date = pd.Timestamp(year=w_copy.index.year.min(), month=4, day=30)

# Calculate daily GDU accumulation
w_copy['Cumulative GDU'] = w_copy.groupby(w_copy.index.year)['gdu'].cumsum()
filtered_w_copy = w_copy[w_copy['Cumulative GDU'] <= 2600]

# Determine the range of GDU accumulation for all years
gdu_by_day = filtered_w_copy.groupby(filtered_w_copy.index.dayofyear)['Cumulative GDU']
gdu_min = gdu_by_day.min()
gdu_max = gdu_by_day.max()
gdu_mean = gdu_by_day.mean()

# Create a date range for the x-axis labels dynamically based on the latest date of reaching 2600 GDU
end_date = filtered_w_copy.index.max()
days_of_year = pd.date_range(start=planting_date, end=end_date, freq='D').dayofyear

# Filter out the dates before the planting date
gdu_min = gdu_min[gdu_min.index >= planting_date.dayofyear]
gdu_max = gdu_max[gdu_max.index >= planting_date.dayofyear]
gdu_mean = gdu_mean[gdu_mean.index >= planting_date.dayofyear]

# Create the plot
plt.figure(figsize=(12, 6))

# Plot the range of all years in shaded gray
plt.fill_between(gdu_min.index, gdu_min, gdu_max, color='gray', alpha=0.3, label='GDU Range (All Years)')

# Plot the average GDU accumulation in dark green
plt.plot(gdu_mean.index, gdu_mean, color='darkgreen', label='Average GDU (All Years)')

# Format the x-axis to show month/day
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
plt.gca().xaxis.set_minor_locator(mdates.WeekdayLocator())

# Set axis labels and title
plt.xlabel('Date')
plt.ylabel('Cumulative GDU')
plt.title('GDU Accumulation for Growing Corn (2600 GDU Target)')
plt.axhline(y=2600, color='r', linestyle='--', label='2600 GDU Target')
plt.legend()

plt.show()


## **Problem 3:** 
Consider the soybean yield file (gott east93 2012 harvest.csv). The rows in the file are sequential measurements made by the mass flow sensor in a combine harvester. Each row includes longitude, latitude, speed, and wet and dry yields measured in bushels per acre and pounds per acre. There are other columns as well.  

1.  Plot the dry yield map to check how it all looks.  
2.  If you wanted to turn the yield rate data into raw yield (i.e., either bushels or pounds), what
    transformation would you have to make?  
3.  Looking at the yield map you should see various passes through the field. Write some code to
    extract the time series corresponding to each pass through the field.  
4.  In each time series estimate the mean, variance, and correlation from sample to sample.
5.  If we thought of each pass as the result of a separate treatment, how could we apply ANOVA to analyze the         hypothesis that all passes have the same mean? Do it and report the result.  

<span style="color:#fff; font-family: 'Babas neue'; font-size: 1.5em;">Solution part 2</span>  

converting bu/ac to bushels for soybean  
1 bushel of soybean is 60 lbs.  
<span style="color:#fff; font-family: 'Babas neue'; font-size: 1.5em;">yield (bu/ac) = $43,560 \times  \frac{(m \times t)}{(d \times w \times p)} \times \frac{100-MC_{harvest}}{100-MC_{market}}$</span>  

m = mass flow rate estimated from the impact plate sensor (lb/sec)  
t = Logging interval of the yield monitoring system (sec)  
d = distance traveled between logged data points (ft)  
w = header cut width setting (ft)  
p = grain density or test weight (lb/bu), 60 for soybean  
$MC_{harvest}$ = moisture content measurement from the yield monitor moisture sensor (%)  
$MC_{market}$ = marketable moisture content (%)  
43,560 = conversion from $ft^2$ to acres



<span style="color:#fff; font-family: 'Babas neue'; font-size: 1.5em;">Solution part 1,3,4, and 5</span>  


In [ ]:
yield_gott93=pd.read_csv("Prelim_figs/gott_east93_2012_harvest.csv",encoding='unicode_escape')
yield_gott93.columns = yield_gott93.columns.str.replace(' ', '') 

yield_gott93.head()
yield_gott93.shape

In [ ]:
gdf = gpd.GeoDataFrame(
    yield_gott93, geometry=gpd.points_from_xy(yield_gott93.Longitude, yield_gott93.Latitude), crs="EPSG:4326"
)
soybean_gott=gdf.to_crs("EPSG:26916")

In [ ]:
soybean_gott.plot("EstimatedVolume(Dry)(bu/ac)",cmap="Greens", markersize=0.3,legend=True,legend_kwds={"label": "bu/ac","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
plt.title("Dry volume measured from Soybean harvest in 2011")
print("Shape: ",soybean_gott.shape)
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)


In [ ]:
soybean_gott.plot("CropFlow(Mass)(lb/s)",cmap="Greens",markersize=0.3, legend=True,legend_kwds={"label": "lb/s","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
plt.title("Crop flow mass measured in combine during Soybean harvest in 2011")
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)

Soybean in 60 lbs per bushel.

In [ ]:
soybean_gott.plot("Speed(mph)",cmap="Reds",markersize=0.3, legend=True,legend_kwds={"label": "mph","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
plt.title("Speed measured during Soybean harvest in 2011")
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)

In [ ]:
soybean_gott.plot("Moisture(%)",cmap="Blues",markersize=0.3, legend=True,legend_kwds={"label": "%","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
plt.title("Moisture measured during Soybean harvest in 2011")
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)

In [ ]:
soybean_gott.plot("Elevation(ft)",cmap="terrain",markersize=0.3, legend=True,legend_kwds={"label": "feet","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
plt.title("Elevation measured during Soybean harvest in 2011")
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)

In [ ]:
soybean_gott.plot("DifferentialValid",cmap="binary", markersize=0.3,legend=True, zorder=20)
plt.title("Differential Validity measured during Soybean harvest in 2011")
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)

In [ ]:
soybean_gott.plot("SwathWidth(ft)",cmap="Set2",markersize=0.3, legend=True,legend_kwds={"label": "feet","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
plt.title("Swath Width measured during Soybean harvest in 2011")
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)

In [ ]:

def calculate_initial_compass_bearing(pointA, pointB):
    import math
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    #use this if the point geometry is in lat\lon decimal degrees
    # pointA = (pointA1.x, pointA1.y)
    # pointB = (pointB1.x, pointB1.y)


    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing
    #return initial_bearing


In [ ]:

data=[[40.99583,-86.17708],[40.98973,-86.18052],[40.99005,-86.17399]]
df=pd.DataFrame(data, columns=["Lat","Lon"])
df["bearing"]=np.nan
for i in range(len(df["Lat"])-1):
    point1=(df["Lat"][i], df["Lon"][i])    
    point2=(df["Lat"][i+1], df["Lon"][i+1])
    print(point1,point2)
    df["bearing"][i]=calculate_initial_compass_bearing(point1,point2)
    print(df["bearing"][i])
df


In [ ]:
X= df['Lat']
Y = df['Lon']
U = np.cos(df["bearing"]) 
V = np.sin(df["bearing"])
print(U,V)
uvis=U*(np.log(df["bearing"]+10)/np.log(2))
vvis=V*(np.log(df["bearing"]+10)/np.log(2))

uvisA= uvis*(100/df["bearing"].max())
vvisA= vvis*(100/df["bearing"].max())
fig, ax1 = plt.subplots(1)
q=ax1.quiver(X,Y,uvisA,vvisA, zorder=21)

In [ ]:
soybean_gott["bearing"]=np.nan
for i in range(len(soybean_gott["geometry"])-1):
    point1=(soybean_gott["Latitude"][i], soybean_gott["Longitude"][i])    
    point2=(soybean_gott["Latitude"][i+1], soybean_gott["Longitude"][i+1])
    soybean_gott["bearing"][i]=calculate_initial_compass_bearing(point1,point2)

In [ ]:
soybean_gott.head(2)

In [ ]:
# gott93 = folium.Map(location=[40.991725, -86.182159], tiles="CartoDB Positron", zoom_start=12)
# gott93

In [ ]:
# yp_gjson = folium.features.GeoJson(soybean_gott, name="Gott 93 yield")
# yp_gjson.add_to(gott93)
# gott93

In [ ]:
soybean_gott_resampled=soybean_gott.sample(frac=0.8)

In [ ]:
X= soybean_gott_resampled['geometry'].x
Y = soybean_gott_resampled['geometry'].y
U = np.cos(soybean_gott_resampled["bearing"]) 
V = np.sin(soybean_gott_resampled["bearing"])
#print(U,V)
uvis=U*(np.log(soybean_gott_resampled["bearing"]+10)/np.log(2))
vvis=V*(np.log(soybean_gott_resampled["bearing"]+10)/np.log(2))

uvisA= uvis*(100/soybean_gott_resampled["bearing"].max())
vvisA= vvis*(100/soybean_gott_resampled["bearing"].max())
myax=soybean_gott_resampled.plot("Speed(mph)",cmap="Reds",markersize=0.5, legend=True,legend_kwds={"label": "mph","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
ax1=myax
q=ax1.quiver(X,Y,uvisA,vvisA, zorder=21)

In [ ]:
print(soybean_gott_resampled["bearing"].describe())


In [ ]:
from shapely.geometry import Polygon
exSite1_latlon=[[-86.17915, 40.99286],[-86.17822,40.9929],[-86.17956,40.98973],[-86.18045,40.98973],[-86.17915,40.99286]]
polygon_geom=Polygon(exSite1_latlon)
exSite1=gpd.GeoDataFrame(crs="EPSG:4326", geometry=[polygon_geom])
exSite1_UTM=exSite1.to_crs(crs="EPSG:26916")
exSite1_UTM.plot()

In [ ]:
from shapely.geometry import Polygon
exSite2_latlon=[[-86.17448, 40.99478],[-86.17375,40.99469],[-86.17583,40.98974],[-86.17671,40.98975],[-86.17448, 40.99478]]
polygon_geom=Polygon(exSite2_latlon)
exSite2=gpd.GeoDataFrame(crs="EPSG:4326", geometry=[polygon_geom])
exSite2_UTM=exSite2.to_crs(crs="EPSG:26916")
exSite2_UTM.plot()

In [ ]:

myax=soybean_gott.plot("YieldMass(Dry)(lb/ac)",cmap="Greens", markersize=0.3,legend=True,legend_kwds={"label": "lb/ac","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30}, zorder=20)
plt.title("Dry Yield measured from Soybean harvest in 2011")
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5, alpha=0.3)
#myax=ax1
exSite1_UTM.plot(ax=myax, color="None", edgecolor="blue",zorder=21 )
exSite2_UTM.plot(ax=myax, color="None", edgecolor="deeppink",zorder=21 )


In [ ]:
print("Shape: ",soybean_gott.shape)


In [ ]:
soybean_gott_site1=soybean_gott.clip(exSite1_UTM.buffer(1.5))
bins=6
colors_yield = plt.cm.brg(np.linspace(0.55, 1, bins+1))
yield_map = colors.LinearSegmentedColormap.from_list('yield_map', colors_yield)
soybean_gott_site1.plot("EstimatedVolume(Dry)(bu/ac)",cmap=yield_map,markersize=0.5, legend="True",legend_kwds={"label": "bu/ac","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30})

In [ ]:
print("Shape: ",soybean_gott_site1.shape)


In [ ]:
def shapecreator(bboxfieldShapeUTM):
    clipper = box(bboxfieldShapeUTM[0],bboxfieldShapeUTM[1],bboxfieldShapeUTM[2],bboxfieldShapeUTM[3])
    fieldClipper = gpd.GeoDataFrame(index=[0], geometry=[clipper])
    bins=6
    colors_yield = plt.cm.brg(np.linspace(0.55, 1, bins+1))
    yield_map = colors.LinearSegmentedColormap.from_list('yield_map', colors_yield)
    ax=soybean_gott_site1.plot("EstimatedVolume(Dry)(bu/ac)",cmap=yield_map,markersize=0.5, legend="True",legend_kwds={"label": "bu/ac","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30})
    fieldClipperRotated=fieldClipper.rotate(angle=-17,origin=(float(bboxfieldShapeUTM[0]-5),float(bboxfieldShapeUTM[1])))
    fieldClipperRotated.plot(ax=ax,color="None",edgecolor="deeppink")
    return fieldClipperRotated

In [ ]:
soybean_gott_site1=soybean_gott_site1[soybean_gott_site1["EstimatedVolume(Dry)(bu/ac)"] < soybean_gott_site1["EstimatedVolume(Dry)(bu/ac)"].quantile(0.999)]
soybean_gott_site1.shape

In [ ]:
exsite1_shape=exSite1_UTM.buffer(1.5)
exsite1_shape1_bounds=float(exsite1_shape.bounds["minx"].iloc[0]-5),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+7),float(exsite1_shape.bounds["maxy"].iloc[0]+14)
exsite1_shape1=shapecreator(exsite1_shape1_bounds)



In [ ]:
exsite1_shape2_bounds=float(exsite1_shape.bounds["minx"].iloc[0]+7),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+15),float(exsite1_shape.bounds["maxy"].iloc[0]+16)
exsite1_shape2=shapecreator(exsite1_shape2_bounds)

In [ ]:
exsite1_shape3_bounds=float(exsite1_shape.bounds["minx"].iloc[0]+17),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+27),float(exsite1_shape.bounds["maxy"].iloc[0]+16)
exsite1_shape3=shapecreator(exsite1_shape3_bounds)

In [ ]:
exsite1_shape4_bounds=float(exsite1_shape.bounds["minx"].iloc[0]+27),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+37),float(exsite1_shape.bounds["maxy"].iloc[0]+20)
exsite1_shape4=shapecreator(exsite1_shape4_bounds)

In [ ]:
exsite1_shape5_bounds=float(exsite1_shape.bounds["minx"].iloc[0]+37),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+47),float(exsite1_shape.bounds["maxy"].iloc[0]+16)
exsite1_shape5=shapecreator(exsite1_shape5_bounds)

In [ ]:
exsite1_shape6_bounds=float(exsite1_shape.bounds["minx"].iloc[0]+50),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+60),float(exsite1_shape.bounds["maxy"].iloc[0]+20)
exsite1_shape6=shapecreator(exsite1_shape6_bounds)

In [ ]:
exsite1_shape7_bounds=float(exsite1_shape.bounds["minx"].iloc[0]+60),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+70),float(exsite1_shape.bounds["maxy"].iloc[0]+20)
exsite1_shape7=shapecreator(exsite1_shape7_bounds)

In [ ]:
exsite1_shape8_bounds=float(exsite1_shape.bounds["minx"].iloc[0]+70),float(exsite1_shape.bounds["miny"].iloc[0]),float(exsite1_shape.bounds["minx"].iloc[0]+80),float(exsite1_shape.bounds["maxy"].iloc[0]+20)
exsite1_shape8=shapecreator(exsite1_shape8_bounds)


In [ ]:
exSite1_row1=soybean_gott_site1.clip(exsite1_shape1) 
exSite1_row1["Row"]=1
print(exSite1_row1.shape)
exSite1_row2=soybean_gott_site1.clip(exsite1_shape2) 
exSite1_row2["Row"]=2
print(exSite1_row2.shape)
exSite1_row3=soybean_gott_site1.clip(exsite1_shape3) 
exSite1_row3["Row"]=3
print(exSite1_row3.shape)
exSite1_row4=soybean_gott_site1.clip(exsite1_shape4) 
exSite1_row4["Row"]=4
print(exSite1_row4.shape)
exSite1_row5=soybean_gott_site1.clip(exsite1_shape5) 
exSite1_row5["Row"]=5
print(exSite1_row5.shape)
exSite1_row6=soybean_gott_site1.clip(exsite1_shape6) 
exSite1_row6["Row"]=6
print(exSite1_row6.shape)
exSite1_row7=soybean_gott_site1.clip(exsite1_shape7) 
exSite1_row7["Row"]=7
print(exSite1_row7.shape)
exSite1_row8=soybean_gott_site1.clip(exsite1_shape8) 
exSite1_row8["Row"]=8
print(exSite1_row8.shape)
list_rows=[exSite1_row1,exSite1_row2,exSite1_row3,exSite1_row4,exSite1_row5,exSite1_row6,exSite1_row7,exSite1_row8]
experimentSite1=pd.concat(list_rows)
experimentSite1.shape
#print(experimentSite1.head())
#print("the mean of the row 8 in experiment site 1 = ",round(exSite1_row8["EstimatedVolume(Dry)(bu/ac)"].mean(),2))
#print("the variance of the row 8 in experiment site 1 = ",round(pow(exSite1_row8["EstimatedVolume(Dry)(bu/ac)"].std(),2),2))

In [ ]:
round(experimentSite1.groupby("Row")["EstimatedVolume(Dry)(bu/ac)"].mean(),2)

In [ ]:
round(pow(experimentSite1.groupby("Row")["EstimatedVolume(Dry)(bu/ac)"].std(),2),2)

In [ ]:
experimentSite1["Row"].describe()

In [ ]:
site=experimentSite1.groupby("Row")
#site.plot("Estimated Volume (Dry)(bu/ac)")
for key, item in site:
    print(item.shape)
    #print(site.get_group(key), "\n\n")

In [ ]:
for key, item in site:
    print(key)
    print(site.get_group(key).shape)
site.get_group(1).shape

In [ ]:
list_corr=[]
for row in range(1,9):
    print(site.get_group(row).shape)
    list_corr.append(round(site.get_group(row)["EstimatedVolume(Dry)(bu/ac)"].autocorr(lag=1),2))
print(list_corr) 
corr_arr=np.array(list_corr)
fig, ax = plt.subplots()
x=np.arange(1,9)
ax=plt.plot(x,corr_arr)
plt.xticks(x,("row1","row2","row3","row4","row5","row6","row7","row8"))
plt.ylabel("lag 1 correlation")


In [ ]:
list_corr=[]
for row in range(1,9):
    list_corr.append(round(site.get_group(row)["EstimatedVolume(Dry)(bu/ac)"].mean(),2))
print(list_corr) 
corr_arr=np.array(list_corr)
fig, ax = plt.subplots()
x=np.arange(1,9)
ax=plt.plot(x,corr_arr)
plt.xticks(x,("row1","row2","row3","row4","row5","row6","row7","row8"))
plt.ylabel("mean yield (bu/ac)")

In [ ]:
list_corr=[]
for row in range(1,9):
    list_corr.append(round(pow(site.get_group(row)["EstimatedVolume(Dry)(bu/ac)"].std(),2),2))
print(list_corr) 
corr_arr=np.array(list_corr)
fig, ax = plt.subplots()
x=np.arange(1,9)
ax=plt.plot(x,corr_arr)
plt.xticks(x,("row1","row2","row3","row4","row5","row6","row7","row8"))
plt.ylabel("variance of yield")

In [ ]:
site1_treatment=[]
treatmentList=["F","M","F","M","F","M","F","M"]
for row in range(1,9):
    df=site.get_group(row)
    print(df.shape)
    df["treatment"]=treatmentList[row-1]
    site1_treatment.append(df)
site1_treatment_df=pd.concat(site1_treatment)

    

In [ ]:
site1_treatment_df=site1_treatment_df.rename(columns={"EstimatedVolume(Dry)(bu/ac)": "EstimatedVolume"})
experiment_dict={1:"Ex1",2:"Ex1",3:"Ex2",4:"Ex2",5:"Ex3",6:"Ex3",7:"Ex4",8:"Ex4"}
site1_treatment_df["Replicate"]=site1_treatment_df["Row"].map(experiment_dict)



In [ ]:
site1_treatment_df["Row"].value_counts()

In [ ]:
dfanova=site1_treatment_df[["EstimatedVolume","treatment","Replicate"]].copy()
dfanova.shape

In [ ]:
siteTreatment=dfanova.groupby("Replicate")
siteTreatment.get_group("Ex1").shape

In [ ]:
def experimentDesigner(experimentlist,treatmentList):
    import matplotlib as mpl
    import matplotlib.patches as mpatches

    mpl.rcParams['hatch.linewidth'] = 0.5
    experiment_dict={"Ex1":"deeppink","Ex2":"blue","Ex3":"Red","Ex4":"Black"}
    treatment_dict={"F":'xx',"M":'oo'}
    bins=6
    colors_yield = plt.cm.brg(np.linspace(0.55, 1, bins+1))
    yield_map = colors.LinearSegmentedColormap.from_list('yield_map', colors_yield)
    ax=soybean_gott_site1.plot("EstimatedVolume(Dry)(bu/ac)",cmap=yield_map,markersize=0.5, legend="True",legend_kwds={"label": "bu/ac","fmt": "{:.1f}", "orientation": "vertical", 'shrink':0.5, 'aspect':30})
    exsite1_shape1.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[0]], hatch = treatment_dict[treatmentList[0]])
    exsite1_shape2.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[0]], hatch = treatment_dict[treatmentList[1]])
    exsite1_shape3.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[1]], hatch = treatment_dict[treatmentList[2]])
    exsite1_shape4.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[1]], hatch = treatment_dict[treatmentList[3]])
    exsite1_shape5.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[2]], hatch = treatment_dict[treatmentList[4]])
    exsite1_shape6.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[2]], hatch = treatment_dict[treatmentList[5]])
    exsite1_shape7.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[3]], hatch = treatment_dict[treatmentList[6]])
    exsite1_shape8.plot(ax=ax,color="None",edgecolor=experiment_dict[experimentlist[3]], hatch = treatment_dict[treatmentList[7]])
    plt.title("Experiment design at Experimental Site 1")
    rect1 = mpatches.Patch( facecolor="None",edgecolor=experiment_dict["Ex1"],label='Rep 1')
    rect2= mpatches.Patch( facecolor="None",edgecolor=experiment_dict["Ex2"],label='Rep 2')
    rect3 = mpatches.Patch(facecolor="None",edgecolor=experiment_dict["Ex3"],label='Rep 3')
    rect4 = mpatches.Patch(facecolor="None",edgecolor=experiment_dict["Ex4"],label='Rep 4')
    rect5 = mpatches.Patch(edgecolor= "black",facecolor="None",hatch='xx',label='Trt 1')
    rect6 = mpatches.Patch(edgecolor= "black",facecolor="None",hatch='oo',label='Trt 2')
    ax.legend(handles = [rect1,rect2,rect3,rect4,rect5,rect6],loc=4)






### I have designed an experiment too compare effect of fertlizer(trt1) vs Manure(trt2) on soybean. Experiment site 1 was chosen because of its least varibility in all other factors during yield collection. The experiment has 4 replicates which are placed side by side with no spatial randomization.

In [ ]:
experimentlist=["Ex1","Ex2","Ex3","Ex4"]

experimentDesigner(experimentlist,treatmentList)

In [ ]:
siteTreatment=dfanova.groupby("Replicate")
for ex in experimentlist:
        df=pd.DataFrame()
        df=siteTreatment.get_group(ex)
        print(df.shape)
        #print(ex)
        # the "C" in C(drainageclabbrvt) is defining it as a categorical variable. hence it is very important here. 
        model = ols('EstimatedVolume ~ C(treatment)', data=df).fit()
        #print("model parameters",model.summary())
        # Perform one-way ANOVA, typ=2 is for both direction testin gof hypothesis.
        anova_results = sm.stats.anova_lm(model, typ=2)
        print(anova_results)

### At $\alpha =0.05$ the treament mean is significant between Manure and fertlizer in all 4 replicates. However their effect size is small.

In [ ]:
def cohend(data1, data2):
    from numpy import mean
    from numpy import var
    from math import sqrt
	# calculate the size of samples
    n1, n2 = len(data1), len(data2)
	# calculate the variance of the samples
    s1, s2 = var(data1, ddof=1), var(data2, ddof=1)
	# calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
	# calculate the means of the samples
    u1, u2 = mean(data1), mean(data2)
	# calculate the effect size
    return (u1 - u2) / s
for row in range(1,8):
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df=site.get_group(row)
    #print(df.head())
    d1=df["EstimatedVolume(Dry)(bu/ac)"]
    df1=site.get_group(row+1)
    d2=df1["EstimatedVolume(Dry)(bu/ac)"]
    effectsize=cohend(d1,d2)
    print("effect size:  ", effectsize)

In [ ]:
import statsmodels.stats.power as smp

# Set parameters
effect_size = 0.2  # Desired effect size (e.g., Cohen's f)
alpha = 0.05       # Significance level
k_groups = 2       # Number of groups in the ANOVA
nobs = 568        # Total sample size (across all groups)

# Create an FTestAnovaPower object
power_analysis = smp.FTestAnovaPower()

# Calculate power
power = power_analysis.power(effect_size, nobs, alpha, k_groups)

print(f"Power: {power}")

In [ ]:
"""
eta_squared (float): ANOVA effect size (eta-squared).

k (int): Number of groups

n (int): Sample size per group. Groups are assumed to be balanced (i.e. same sample size).

power (float) :Test power (= 1 - type II error).

alpha (float) : Significance level (type I error probability). The default is 0.05."""
from pingouin import power_anova
print('n: %.4f' % power_anova(eta_squared=0.2237,k=2,power=.99, alpha=0.01))